In [55]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
import nltk
import pandas as pd
import re
from PIL import Image, ImageFilter
from PIL import Image, ImageEnhance
import os
import glob

In [56]:
# !pip install datefinder

In [57]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [58]:
# read df from files/isp_data.csv
df = pd.read_csv('../files/isp_data.csv')

In [59]:
# read 

Skipping image 1.png as it's already in the DataFrame.
Skipping image 10.png as it's already in the DataFrame.
Skipping image 11.png as it's already in the DataFrame.
Skipping image 12.png as it's already in the DataFrame.
Skipping image 2.png as it's already in the DataFrame.
Skipping image 3.png as it's already in the DataFrame.
Skipping image 4.png as it's already in the DataFrame.
Skipping image 5.png as it's already in the DataFrame.
Skipping image 6.png as it's already in the DataFrame.
Skipping image 7.png as it's already in the DataFrame.
Skipping image 8.png as it's already in the DataFrame.
Skipping image 9.png as it's already in the DataFrame.


In [61]:
# df

In [63]:
df

,image_path,ISP,date,amount,year,month
0,../Data/Internet_bills_dialog/Images\1.png,Dialog,2022-11-04,2890.44,2022,11
1,../Data/Internet_bills_dialog/Images\10.png,Dialog,2023-01-04,2800.00,2023,1
2,../Data/Internet_bills_dialog/Images\11.png,Dialog,2023-01-26,4200.00,2023,1
3,../Data/Internet_bills_dialog/Images\12.png,Dialog,2023-01-26,4000.00,2023,1
4,../Data/Internet_bills_dialog/Images\2.png,Dialog,2022-03-23,500.00,2022,3
5,../Data/Internet_bills_dialog/Images\3.png,Dialog,2022-05-31,800.00,2022,5
6,../Data/Internet_bills_dialog/Images\4.png,Dialog,2022-07-04,2226.56,2022,7
7,../Data/Internet_bills_dialog/Images\5.png,Dialog,2022-07-15,800.00,2022,7
8,../Data/Internet_bills_dialog/Images\6.png,Dialog,2022-08-07,800.00,2022,8
9,../Data/Internet_bills_dialog/Images\7.png,Dialog,2022-08-28,4387.23,2022,8


In [64]:
# save the dataframe as a csv file
df.to_csv('../files/isp_data.csv', index=False)

In [65]:
monthly_payments = df.groupby(['ISP','year', 'month'])['amount'].sum().reset_index()

monthly_payments = pd.DataFrame(monthly_payments)

monthly_payments


,ISP,year,month,amount
0,Dialog,2022,3,500.00
1,Dialog,2022,5,800.00
2,Dialog,2022,7,3026.56
3,Dialog,2022,8,5187.23
4,Dialog,2022,11,7790.44
5,Dialog,2023,1,11000.00
